In [1117]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [1118]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display=Display(visible=0,size=(400,300))
    display.start()

In [1119]:
# default_exp fastai.loop

In [1120]:
# export
# Python native modules
import os,sys
from copy import deepcopy,copy
from typing import *
import types
import logging
import inspect
from itertools import chain
from functools import partial
# Third party libs
from fastcore.all import *
import numpy as np
# Local modules

_logger=logging.getLogger(__name__)

# Loop
> fastrl concept of generic loop objects. 

The goal for Loops is to make it easy to customize, and know how sections of code connects
to other parts.

### Why do we need this?
We have identified at least 3 different kinds of loops already:

    Learner (training)
    Source/Gym (Data Access)
    Agent (How an AI takes in data, generates actions)

### What is a loop?

    It should be capable of containing inner loops. 
    It should self-describe its structure. 
    It should be easy to know which parts of the loop are taking long/short amounts of time.
    It should be flexible in state modification.
    It should alternatively make it easy show what fields are being changed at what points in time.

A Loop will act as a compiled structure. The actual result will be a compiled list of nodes that reference the original loop.

In [1121]:
# export
EVENT_ORDER_MAPPING={}
PREFIXES=['before_','on_','after_','failed_','finally_']

def _is_event(o): return issubclass(o.__class__,Event)
def _last_element(ls): return ls[-1]
def _grab_full_name(o:'Event'): return o.full_name

class EventException(Exception):pass

class Event(object):
    def __init__(self,
                 function:Callable,
                 loop=None,
                ):
        store_attr()
        # We set the order over the entire Loop definition
        self.outer_name=function.__module__+'.'+function.__qualname__.split('.')[0]
        if self.outer_name not in EVENT_ORDER_MAPPING: self.order=1
        else: self.order=EVENT_ORDER_MAPPING[self.outer_name]
        EVENT_ORDER_MAPPING[self.outer_name]=self.order+1
        
        self.full_name=function.__module__+'.'+function.__qualname__
            
        if self.name.startswith('_') or not any(self.name.startswith(pre) for pre in PREFIXES):
            raise EventException(f'{self.name} needs to start with any {PREFIXES}')
            
        self.cbs=L()
        
    def set_cbs(self,cbs=None):
        if cbs is not None:
            self.cbs=L((cb() if isinstance(cb, type) else cb) for cb in L(cbs) if hasattr(cb,self.name))
            

    def __call__(self,*args,**kwargs): 
        ret=self.function(self.loop,*args,**kwargs)
        for cb in self.cbs: cb_ret=getattr(cb,self.name)()
        return ret

    def __lt__(self,o:'Event'): return self.order<o.order
    @property
    def name(self): return self.function.__name__
    @property
    def prefix(self): return self.name.split('_')[0]+'_'
    @property
    def postfix(self): return '_'.join(self.name.split('_')[1:])
    def __repr__(self): return self.full_name
    
    def show(self,n_tab=0,n_cbs=False,include_cbs=False,**kwargs):
        kwargs=merge(dict(n_cbs=n_cbs,include_cbs=include_cbs),kwargs)
        tab='\t' if self.loop is None or self.loop.root_loop is None else self.loop.root_loop.tab
        event=n_tab*tab+str(self)
        if self.cbs: event+=f' #{len(self.cbs)}'
        if include_cbs: 
            for cb in self.cbs:
                event+='\n'+(n_tab*tab)+cb.show(n_tab=n_tab+1,**kwargs)
        return event
    
def isrelevent(loop_or_cb,event:Event):
    if isinstance(event,(list,L)): return False
    return event.full_name in loop_or_cb.call_on.map(_grab_full_name)


event=Event

In [1122]:
def on_test():pass

decorated_on_test=Event(on_test)

In [1123]:
decorated_on_test.full_name

'__main__.on_test'

In [1124]:
class B():
    @event
    def on_test_2_a(self):pass

In [1125]:
B.on_test_2_a.full_name

'__main__.B.on_test_2_a'

In [1126]:

class A(object):
    parent_loop,root_loop=None,None
    
    @classmethod
    def events(cls,instance=None,instantiate=False):
        loop=instance if instance is not None else (cls() if instantiate else cls)
        events=L(inspect.getmembers(loop)).map(_last_element).filter(_is_event)
        for o in events: o.loop=loop
        return events
    
    @event
    def before_test_c(self):print('before_test_c')
    @event
    def on_test_c(self):print('on_test_c')
    @event
    def after_test_c(self):print('after_test_c')
    @event
    def after_test_c(self):print('after_test_c')
    @event
    def on_test_b(self):print('on_test_b')
    @event
    def on_test_a(self):print('on_test_a')

In [1127]:
A().before_test_c()

before_test_c


Notes:
- Having a section object might make this overal loop management better
    - remaining stuff, the Section object needs to handle lists and execute those also.
    - with this in mind, using the section I think will make the looping run much cleaner.

In [1128]:
# export
def _grab_postfix(e:Union[Event,list],previous_event:dict=None): 
    if not isinstance(e,Event):
        if not previous_event: 
            raise SectionException(f'{e} doesnt have an event tied to it.')
        else:
            return previous_event['last_event']
    if previous_event is not None:
        previous_event['last_event']=e.postfix
    return e.postfix

def _grab_prefix(o:Event): return o.prefix
def _default_raise(): raise 
def _event2dict(e:Union[Event,list,L],previous_event:dict):
    if not isinstance(e,Event):
        if not previous_event: 
            raise SectionException(f'{e} doesnt have an event tied to it.')
        else:
            return (previous_event['last_event']+'inner',e)
    
    previous_event['last_event']=e.prefix
    return (e.prefix,e)

def _noop_event_pair(k):
    return ((k,noop if k!='failed_' else _default_raise),(k+'inner',[]))


class SectionException(Exception):pass

class Section(object):
    
    def __init__(self,events,loop=None):
        self.loop=loop
        default_events=L(PREFIXES).map(_noop_event_pair)
        default_events=L(chain.from_iterable(default_events))
        previous_event={}
        events=[o for o in events if not isinstance(o,(list,L)) or len(o)!=0]
        self.events=merge(
            dict(default_events),
            dict(L(events).map(_event2dict,previous_event=previous_event))
        )
        print(self,self.loop,self.events)
        
    def __repr__(self): return str(self.__class__.__name__)
    def __len__(self): return len(L(self.events.values()).filter(_is_event)) 

    @delegates(Event.show)
    def show(self,n_tab=0,n_events=False,include_events=False,include_defaults=False,**kwargs):
        kwargs=merge(dict(n_events=n_events,
                          include_events=include_events,
                          include_defaults=include_defaults),kwargs)
        tab='\t' if self.loop is None else self.loop.root_loop.tab
        section=n_tab*tab+str(self)
        if n_events:section+=f' {len(self)} events'
        if include_events:
            for event in self.events.values(): 
                if isinstance(event,(list,L)):
                    for o in event:
                        section+='\n'+(n_tab*tab)+o.show(n_tab=n_tab+1,**kwargs)
                elif event in [_default_raise,noop] and include_defaults:
                    section+='\n'+((n_tab+1)*tab)+str(event)
                elif event not in [_default_raise,noop]:
                    section+='\n'+(n_tab*tab)+event.show(n_tab=n_tab+1,**kwargs)
        return section

    def run(self):
        print(self,self.loop,self.events)
        try:
            self.events['before_']()
            for o in self.events['before_inner']: o.run()
            self.events['on_']()
            for o in self.events['on_inner']: o.run()
            
            self.events['after_']()
            for o in self.events['after_inner']: o.run()
        except Exception as ex:
            try:     
                self.events['failed_']()
                raise
            finally: 
                for o in self.events['failed_inner']: o.run()
        finally:
            self.events['finally_']()
            for o in self.events['finally_inner']: 
                print(o)
                o.run()
    
    @classmethod
    def from_events(cls,events:List[Event],parent_loop=None):
        previous_event={}
        event_groups=groupby(events,partial(_grab_postfix,previous_event=previous_event))
        return [cls(o,parent_loop) for o in event_groups.values()]

In [1129]:
Section.from_events(A.events())[0].events

Section None {'before_': __main__.A.before_test_c, 'before_inner': [], 'on_': __main__.A.on_test_c, 'on_inner': [], 'after_': __main__.A.after_test_c, 'after_inner': [], 'failed_': <function _default_raise at 0x7f6f8b87aee0>, 'failed_inner': [], 'finally_': <function noop at 0x7f6fc43f25e0>, 'finally_inner': []}
Section None {'before_': <function noop at 0x7f6fc43f25e0>, 'before_inner': [], 'on_': __main__.A.on_test_a, 'on_inner': [], 'after_': <function noop at 0x7f6fc43f25e0>, 'after_inner': [], 'failed_': <function _default_raise at 0x7f6f8b87aee0>, 'failed_inner': [], 'finally_': <function noop at 0x7f6fc43f25e0>, 'finally_inner': []}
Section None {'before_': <function noop at 0x7f6fc43f25e0>, 'before_inner': [], 'on_': __main__.A.on_test_b, 'on_inner': [], 'after_': <function noop at 0x7f6fc43f25e0>, 'after_inner': [], 'failed_': <function _default_raise at 0x7f6f8b87aee0>, 'failed_inner': [], 'finally_': <function noop at 0x7f6fc43f25e0>, 'finally_inner': []}


{'before_': __main__.A.before_test_c,
 'before_inner': [],
 'on_': __main__.A.on_test_c,
 'on_inner': [],
 'after_': __main__.A.after_test_c,
 'after_inner': [],
 'failed_': <function __main__._default_raise()>,
 'failed_inner': [],
 'finally_': <function fastcore.imports.noop(x=None, *args, **kwargs)>,
 'finally_inner': []}

In [1130]:
A.events(A())[0]()

after_test_c


In [1131]:
[o.show( 
    n_events=      True,
    include_events=True,
    n_cbs=         True,
    include_cbs=   True) for o in Section.from_events(A.events(A()))]

Section None {'before_': __main__.A.before_test_c, 'before_inner': [], 'on_': __main__.A.on_test_c, 'on_inner': [], 'after_': __main__.A.after_test_c, 'after_inner': [], 'failed_': <function _default_raise at 0x7f6f8b87aee0>, 'failed_inner': [], 'finally_': <function noop at 0x7f6fc43f25e0>, 'finally_inner': []}
Section None {'before_': <function noop at 0x7f6fc43f25e0>, 'before_inner': [], 'on_': __main__.A.on_test_a, 'on_inner': [], 'after_': <function noop at 0x7f6fc43f25e0>, 'after_inner': [], 'failed_': <function _default_raise at 0x7f6f8b87aee0>, 'failed_inner': [], 'finally_': <function noop at 0x7f6fc43f25e0>, 'finally_inner': []}
Section None {'before_': <function noop at 0x7f6fc43f25e0>, 'before_inner': [], 'on_': __main__.A.on_test_b, 'on_inner': [], 'after_': <function noop at 0x7f6fc43f25e0>, 'after_inner': [], 'failed_': <function _default_raise at 0x7f6f8b87aee0>, 'failed_inner': [], 'finally_': <function noop at 0x7f6fc43f25e0>, 'finally_inner': []}


['Section 3 events\n\t__main__.A.before_test_c\n\t__main__.A.on_test_c\n\t__main__.A.after_test_c',
 'Section 1 events\n\t__main__.A.on_test_a',
 'Section 1 events\n\t__main__.A.on_test_b']

In [1132]:
[print(o.show( 
    n_events=      True,
    include_events=True,
    n_cbs=         True,
    include_cbs=   True)) for o in Section.from_events(A.events(A()))]

Section None {'before_': __main__.A.before_test_c, 'before_inner': [], 'on_': __main__.A.on_test_c, 'on_inner': [], 'after_': __main__.A.after_test_c, 'after_inner': [], 'failed_': <function _default_raise at 0x7f6f8b87aee0>, 'failed_inner': [], 'finally_': <function noop at 0x7f6fc43f25e0>, 'finally_inner': []}
Section None {'before_': <function noop at 0x7f6fc43f25e0>, 'before_inner': [], 'on_': __main__.A.on_test_a, 'on_inner': [], 'after_': <function noop at 0x7f6fc43f25e0>, 'after_inner': [], 'failed_': <function _default_raise at 0x7f6f8b87aee0>, 'failed_inner': [], 'finally_': <function noop at 0x7f6fc43f25e0>, 'finally_inner': []}
Section None {'before_': <function noop at 0x7f6fc43f25e0>, 'before_inner': [], 'on_': __main__.A.on_test_b, 'on_inner': [], 'after_': <function noop at 0x7f6fc43f25e0>, 'after_inner': [], 'failed_': <function _default_raise at 0x7f6f8b87aee0>, 'failed_inner': [], 'finally_': <function noop at 0x7f6fc43f25e0>, 'finally_inner': []}
Section 3 events
	__

[None, None, None]

In [1133]:
[o.run() for o in Section.from_events(A.events(A()))]

Section None {'before_': __main__.A.before_test_c, 'before_inner': [], 'on_': __main__.A.on_test_c, 'on_inner': [], 'after_': __main__.A.after_test_c, 'after_inner': [], 'failed_': <function _default_raise at 0x7f6f8b87aee0>, 'failed_inner': [], 'finally_': <function noop at 0x7f6fc43f25e0>, 'finally_inner': []}
Section None {'before_': <function noop at 0x7f6fc43f25e0>, 'before_inner': [], 'on_': __main__.A.on_test_a, 'on_inner': [], 'after_': <function noop at 0x7f6fc43f25e0>, 'after_inner': [], 'failed_': <function _default_raise at 0x7f6f8b87aee0>, 'failed_inner': [], 'finally_': <function noop at 0x7f6fc43f25e0>, 'finally_inner': []}
Section None {'before_': <function noop at 0x7f6fc43f25e0>, 'before_inner': [], 'on_': __main__.A.on_test_b, 'on_inner': [], 'after_': <function noop at 0x7f6fc43f25e0>, 'after_inner': [], 'failed_': <function _default_raise at 0x7f6f8b87aee0>, 'failed_inner': [], 'finally_': <function noop at 0x7f6fc43f25e0>, 'finally_inner': []}
Section None {'befor

[None, None, None]

In [1134]:
# export
def _is_event(o): return issubclass(o.__class__,Event)
def _last_element(ls): return ls[-1]
def _loop2sections(loop,loops,cbs): return loop.sections(loops=loops,cbs=cbs)
def _loop_with_sections(loop,loops,cbs,parent_loop): 
    loop.parent_loop=parent_loop
    return loop.from_sections(loops=loops,cbs=cbs)

class class_or_instancemethod(classmethod):
    "From: https://stackoverflow.com/questions/28237955/same-name-for-classmethod-and-instancemethod"
    def __get__(self, instance, type_):
        descr_get = super().__get__ if instance is None else self.__func__.__get__
        return descr_get(instance, type_)

class Loop(object):  
    call_on,parent_loop,tab=L(),None,'  '
    
    @class_or_instancemethod
    def events(cls_or_self,loops=None,cbs=None):
        events=L(inspect.getmembers(cls_or_self)).map(_last_element)\
                                                 .filter(_is_event)\
                                                 .sorted()
        for o in events: 
            o.loop=cls_or_self
            for cb in L(cbs): cb.loop=cls_or_self
            o.set_cbs(cbs)
        
        events=chain.from_iterable([
            (o,L(ifnone(loops,L()).filter(isrelevent,event=o)\
                                .map(_loop_with_sections,
                                     loops=loops,cbs=cbs,
                                     parent_loop=cls_or_self))) 
            for o in events
        ])
        return events

    @class_or_instancemethod
    def get_sections(cls_or_self,loops=None,cbs=None):
        events=cls_or_self.events(loops=L(loops),cbs=L(cbs))
        sections=Section.from_events(events,cls_or_self)
        return sections
    
    @classmethod
    def from_sections(cls,**kwargs):
        loop=cls()
        loop.sections=loop.get_sections(**kwargs)
        return loop
    
    def run(self,loops=None,cbs=None):
        sections=self.get_sections(loops=loops,cbs=cbs)
        for section in sections:
            section.run()
            
    def __len__(self): return len(self.sections)
    def __repr__(self): return str(self.__class__.__name__)
    @property
    def root_loop(self):
        return self if self.parent_loop is None else self.parent_loop

    @delegates(Section.show)
    def show(self,n_tab=0,n_sections=False,include_sections=False,**kwargs):
        tab=self.root_loop.tab
        kwargs=merge(dict(n_sections=n_sections,include_sections=include_sections),kwargs)
        loop=n_tab*tab+str(self)
        if n_sections:loop+=f' {len(self)} sections'
        if include_sections:
            for section in self.sections: 
                loop+='\n'+(n_tab*tab)
                loop+=section.show(n_tab=n_tab+1,**kwargs)
        return loop

class Outer(Loop):
    @event
    def before_step(self) :  print('before_step')
    @event
    def on_step(self)     :  print('on_step')
    @event
    def after_step(self)  :  print('after_step')
    @event
    def failed_step(self) :  print('failed_step')
    @event
    def finally_step(self):  print('finally_step')
 
    @event
    def before_jump(self) :  print('before_jump')
    @event
    def on_jump(self)     :  print('on_jump')
    @event
    def after_jump(self)  :  print('after_jump')
    @event
    def failed_jump(self) :  print('failed_jump')
    @event
    def finally_jump(self):  print('finally_jump')

list(Outer.events())

[__main__.Outer.before_step,
 (#0) [],
 __main__.Outer.on_step,
 (#0) [],
 __main__.Outer.after_step,
 (#0) [],
 __main__.Outer.failed_step,
 (#0) [],
 __main__.Outer.finally_step,
 (#0) [],
 __main__.Outer.before_jump,
 (#0) [],
 __main__.Outer.on_jump,
 (#0) [],
 __main__.Outer.after_jump,
 (#0) [],
 __main__.Outer.failed_jump,
 (#0) [],
 __main__.Outer.finally_jump,
 (#0) []]

In [1135]:
Outer().get_sections()

Section Outer {'before_': __main__.Outer.before_step, 'before_inner': [], 'on_': __main__.Outer.on_step, 'on_inner': [], 'after_': __main__.Outer.after_step, 'after_inner': [], 'failed_': __main__.Outer.failed_step, 'failed_inner': [], 'finally_': __main__.Outer.finally_step, 'finally_inner': []}
Section Outer {'before_': __main__.Outer.before_jump, 'before_inner': [], 'on_': __main__.Outer.on_jump, 'on_inner': [], 'after_': __main__.Outer.after_jump, 'after_inner': [], 'failed_': __main__.Outer.failed_jump, 'failed_inner': [], 'finally_': __main__.Outer.finally_jump, 'finally_inner': []}


[Section, Section]

In [1136]:
Outer().run()

Section Outer {'before_': __main__.Outer.before_step, 'before_inner': [], 'on_': __main__.Outer.on_step, 'on_inner': [], 'after_': __main__.Outer.after_step, 'after_inner': [], 'failed_': __main__.Outer.failed_step, 'failed_inner': [], 'finally_': __main__.Outer.finally_step, 'finally_inner': []}
Section Outer {'before_': __main__.Outer.before_jump, 'before_inner': [], 'on_': __main__.Outer.on_jump, 'on_inner': [], 'after_': __main__.Outer.after_jump, 'after_inner': [], 'failed_': __main__.Outer.failed_jump, 'failed_inner': [], 'finally_': __main__.Outer.finally_jump, 'finally_inner': []}
Section Outer {'before_': __main__.Outer.before_step, 'before_inner': [], 'on_': __main__.Outer.on_step, 'on_inner': [], 'after_': __main__.Outer.after_step, 'after_inner': [], 'failed_': __main__.Outer.failed_step, 'failed_inner': [], 'finally_': __main__.Outer.finally_step, 'finally_inner': []}
before_step
on_step
after_step
finally_step
Section Outer {'before_': __main__.Outer.before_jump, 'before_

In [1137]:

class Inner(Loop):
    call_on=L(Outer.on_step)#,Outer.after_step,Outer.finally_jump)
    
    @event
    def before_iteration(self) : print('before_iteration')
    @event
    def on_iteration(self)     : print('on_iteration')
    @event
    def after_iteration(self)  : print('after_iteration')
    @event
    def failed_iteration(self) : print('failed_iteration')
    @event
    def finally_iteration(self): print('finally_iteration')
    def run(self,loops=None,cbs=None):
        sections=self.get_sections(loops=loops,cbs=cbs)
        for section in sections:
            section.run()
    
class FailingInner(Loop):
    call_on=L(Inner.finally_iteration)
    
    @event
    def on_force_fail(self):                    
        print('on_force_fail')
        raise Exception
        

In [1138]:
Outer().events

<bound method Loop.events of Outer>

In [1139]:
Outer().get_sections(L(Inner(),FailingInner()))

Section FailingInner {'before_': <function noop at 0x7f6fc43f25e0>, 'before_inner': [], 'on_': __main__.FailingInner.on_force_fail, 'on_inner': [], 'after_': <function noop at 0x7f6fc43f25e0>, 'after_inner': [], 'failed_': <function _default_raise at 0x7f6f8b87aee0>, 'failed_inner': [], 'finally_': <function noop at 0x7f6fc43f25e0>, 'finally_inner': []}
Section Inner {'before_': __main__.Inner.before_iteration, 'before_inner': [], 'on_': __main__.Inner.on_iteration, 'on_inner': [], 'after_': __main__.Inner.after_iteration, 'after_inner': [], 'failed_': __main__.Inner.failed_iteration, 'failed_inner': [], 'finally_': __main__.Inner.finally_iteration, 'finally_inner': [FailingInner]}
Section Outer {'before_': __main__.Outer.before_step, 'before_inner': [], 'on_': __main__.Outer.on_step, 'on_inner': [Inner], 'after_': __main__.Outer.after_step, 'after_inner': [], 'failed_': __main__.Outer.failed_step, 'failed_inner': [], 'finally_': __main__.Outer.finally_step, 'finally_inner': []}
Sectio

[Section, Section]

In [1140]:
class CallbackException(Exception):pass

class Callback(object):
    call_on,loop=None,None
    
    def show(self,n_tab=0,**kwargs):
        tab=self.loop.root_loop.tab if self.loop is not None else '\t'
        return (tab*n_tab)+str(self)

In [1141]:
class OuterCallback(Callback):
    call_on=L(Outer.on_step)
    
    def before_iteration(self)->dict(this=list,that=str):
        print('   OuterCallback called lol')

Outer().get_sections(L(Inner(),FailingInner()),OuterCallback)

Section FailingInner {'before_': <function noop at 0x7f6fc43f25e0>, 'before_inner': [], 'on_': __main__.FailingInner.on_force_fail, 'on_inner': [], 'after_': <function noop at 0x7f6fc43f25e0>, 'after_inner': [], 'failed_': <function _default_raise at 0x7f6f8b87aee0>, 'failed_inner': [], 'finally_': <function noop at 0x7f6fc43f25e0>, 'finally_inner': []}
Section Inner {'before_': __main__.Inner.before_iteration, 'before_inner': [], 'on_': __main__.Inner.on_iteration, 'on_inner': [], 'after_': __main__.Inner.after_iteration, 'after_inner': [], 'failed_': __main__.Inner.failed_iteration, 'failed_inner': [], 'finally_': __main__.Inner.finally_iteration, 'finally_inner': [FailingInner]}
Section Outer {'before_': __main__.Outer.before_step, 'before_inner': [], 'on_': __main__.Outer.on_step, 'on_inner': [Inner], 'after_': __main__.Outer.after_step, 'after_inner': [], 'failed_': __main__.Outer.failed_step, 'failed_inner': [], 'finally_': __main__.Outer.finally_step, 'finally_inner': []}
Sectio

[Section, Section]

In [1142]:
print(Outer.from_sections(
    loops=L(Inner(),FailingInner()),
    cbs=OuterCallback
                   
                   
).show(
    n_sections=      True,
    include_sections=True,
    n_events=        True,
    include_events=  True,
    n_cbs=           True,
    include_cbs=     True,
    include_defaults=True
))

Section FailingInner {'before_': <function noop at 0x7f6fc43f25e0>, 'before_inner': [], 'on_': __main__.FailingInner.on_force_fail, 'on_inner': [], 'after_': <function noop at 0x7f6fc43f25e0>, 'after_inner': [], 'failed_': <function _default_raise at 0x7f6f8b87aee0>, 'failed_inner': [], 'finally_': <function noop at 0x7f6fc43f25e0>, 'finally_inner': []}
Section Inner {'before_': __main__.Inner.before_iteration, 'before_inner': [], 'on_': __main__.Inner.on_iteration, 'on_inner': [], 'after_': __main__.Inner.after_iteration, 'after_inner': [], 'failed_': __main__.Inner.failed_iteration, 'failed_inner': [], 'finally_': __main__.Inner.finally_iteration, 'finally_inner': [FailingInner]}
Section Outer {'before_': __main__.Outer.before_step, 'before_inner': [], 'on_': __main__.Outer.on_step, 'on_inner': [Inner], 'after_': __main__.Outer.after_step, 'after_inner': [], 'failed_': __main__.Outer.failed_step, 'failed_inner': [], 'finally_': __main__.Outer.finally_step, 'finally_inner': []}
Sectio

In [1144]:

# Outer().get_sections(L(Inner(),FailingInner()),OuterCallback)[0].events['on_inner'][0].sections[0].run()

In [1145]:

Outer().run(L(Inner(),FailingInner()),OuterCallback())

Section FailingInner {'before_': <function noop at 0x7f6fc43f25e0>, 'before_inner': [], 'on_': __main__.FailingInner.on_force_fail, 'on_inner': [], 'after_': <function noop at 0x7f6fc43f25e0>, 'after_inner': [], 'failed_': <function _default_raise at 0x7f6f8b87aee0>, 'failed_inner': [], 'finally_': <function noop at 0x7f6fc43f25e0>, 'finally_inner': []}
Section Inner {'before_': __main__.Inner.before_iteration, 'before_inner': [], 'on_': __main__.Inner.on_iteration, 'on_inner': [], 'after_': __main__.Inner.after_iteration, 'after_inner': [], 'failed_': __main__.Inner.failed_iteration, 'failed_inner': [], 'finally_': __main__.Inner.finally_iteration, 'finally_inner': [FailingInner]}
Section Outer {'before_': __main__.Outer.before_step, 'before_inner': [], 'on_': __main__.Outer.on_step, 'on_inner': [Inner], 'after_': __main__.Outer.after_step, 'after_inner': [], 'failed_': __main__.Outer.failed_step, 'failed_inner': [], 'finally_': __main__.Outer.finally_step, 'finally_inner': []}
Sectio

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script('[!02_fastai.loop.old]*.ipynb')
    notebook2html('[!02_fastai.loop.old]*.ipynb')